In [67]:
import json
import os
from openai import AzureOpenAI

config_path = 'config.json'
with open(config_path, 'r') as config_file:
    config = json.load(config_file)

azure_api_key = config['azure_api_key']
azure_api_version = config['azure_api_version']
azure_endpoint = config['azure_endpoint']
deployment_name = config['deployment_name']

client = AzureOpenAI(
    api_key=azure_api_key,  
    api_version=azure_api_version,
    azure_endpoint=azure_endpoint
)

def generate_text(prompt):

    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=200,
        temperature=0.7
    )

    text = response.choices[0].message.content.strip()

    return text

def generate_response(feedback):
    prompt = f"For the given feedback {feedback}, check if it is positive, negative or mixed. Give it under 'Sentiment:' heading, P for positive, N for negative and M for mixed. For positive or (negative and mixed) specify which category or categories (service, food, etc.) was positive or negative under heading 'Categories:' and the severity (1 to 5 range) of the feedback under heading 'Severity:'. For positive severity is 0. Don't write anything else or break format."
    analysis = generate_text(prompt)
    parts = analysis.split("\n")
    for part in parts:
        if part.startswith("Sentiment:"):
            sentiment = part.replace("Sentiment:", "").strip()
        elif part.startswith("Categories:"):
            category = part.replace("Categories:", "").strip()
        elif part.startswith("Severity:"):
            severity = part.replace("Severity:", "").strip()
    if sentiment == 'N':
        prompt_2 = f"For the given feedback {feedback}, generate only a apologetic response with a promise to improve. Don't write anyting else. No email,reply,message response format."
        response = generate_text(prompt_2)
    elif sentiment == 'M':
        prompt_2 = f"For the given feedback {feedback}, generate only a appreciative response and apology with a promise to improve. Don't write anyting else. No email,reply,message response format."
        response = generate_text(prompt_2)
    elif sentiment == 'P':
        prompt_2 = f"For the given feedback {feedback}, generate only a appreciative response. Don't write anyting else. No email,reply,message response format."
        response = generate_text(prompt_2)

                
    return sentiment, response, category, severity




In [68]:
# Example
feedback_p1 = "We recently had them cater for our son’s wedding welcome party at the Alloy hotel. Was very nervous with Indian food catering since all items taste the same with the same masala in most Indian restaurants. They did a fantastic job with the food that was cooked on site in the hotel kitchen. The appetizers (wide selection) were excellent and especially the chat stand with a variety of pani puri, dhai puri and Bhel Puri and not to forget the cool mango lassi to compliment the various other drinks. The main meal was unique with a southIndian flavor with no garlic (on request) and the flavors were not the same masala in all dishes. The butter chicken and lamb dishes were a big hit. The desserts (3 different ones) were very good and especially the young and old alike devoured on the gulab jamuns. Overall the service and food was excellent and will highly recommend talking to the owner Karthik to get things going. Got value for our money."
feedback_p2 = "Very great Indian restaurant. Stopped by this place during my business trip. Loved the atmosphere and the customer service. I tried the authentic chicken korma and mutton gravy with malabar poratta. Very great taste and loved how flaky poratta was. I will be visiting pa again and very excited to try new dishes at this place. I'm a big fan of biryani, that's in my wishlist when I visit this place next time."

feedback_n1 = "I would give minus rating if possible, We order fish biryani and dosa. In Biryani they mix chicken with fish, though I clearly ask them for fish biryani.  We didn't eat the biryani, still paid for it. They were even not apologetic for it. One of the worst experiences."
feedback_n2 = "The food was meh. No seasoning whatsoever. I am guessing a piece of cardboard will taste better than the pomfret fish fry, I’d rather eat the cardboard."

feedback_m1 = "Very nice staff, food is less spicy which is a good thing for me.  Have been here few times and i  liked all dishes except the dosa and its chutney - maybe this one is  just not my thing"
feedback_m2 = "May be this was just one off experience - visited this place after the take out experience was splendid. But dine-in was just above average. Some of the dishes lacked salt. A friend had a similar experience as well."

sentiment, response, category, severity = generate_response(feedback_m2)
print("Response for feedback:")
print(response)
print("Sentiment for feedback: ",sentiment)
print("Category of negative/positive area in feedback: ", category)
print("Severity of feedback (0 positive, 1 to 5 negative/mixed): ", severity)

Response for feedback:
Thank you for sharing your feedback. We apologize for the inconsistency during your dine-in experience. We appreciate your input and will strive to improve our service and food quality to ensure a better experience next time.
Sentiment for feedback:  M
Category of negative/positive area in feedback:  food (negative)
Severity of feedback:  3
